# Replicating object-oriented results with procedural

In [1]:
import pvlib
from pandas import read_csv

## Setup

In [2]:
latitude = 42.23282
longitude = -8.72264
altitude = 80
surface_tilt = 0.0
surface_azimuth = 0.0
module = {'A_c': 1.7, 'Adjust': 14.047909, 'BIPV': 'N', 'Bifacial': 0, 'Date': '1/3/2019', 'I_L_ref': 4.798116, 'I_mp_ref': 4.31, 'I_o_ref': 1.366077e-09, 'I_sc_ref': 4.78, 'Length': 1.602, 'N_s': 96, 'PTC': 181.6, 'R_s': 0.793104, 'R_sh_ref': 209.272705, 'STC': 199.984, 'T_NOCT': 42.4, 'Technology': 'Mono-c-Si', 'V_mp_ref': 46.4, 'V_oc_ref': 57.4, 'Version': 'SAM 2018.11.11 r2', 'Width': 1.061, 'a_ref': 2.618532, 'alpha_sc': 0.004254, 'beta_oc': -0.214676, 'gamma_r': -0.476}
inverter = {'C0': -4.1e-05, 'C1': -9.1e-05, 'C2': 0.000494, 'C3': -0.013171, 'CEC_Type': 'Utility Interactive', 'Idcmax': 6.489715, 'Mppt_high': 50.0, 'Mppt_low': 30.0, 'Paco': 250.0, 'Pdco': 259.588593, 'Pnt': 0.075, 'Pso': 2.089607, 'Vac': '208', 'Vdcmax': 50.0, 'Vdco': 40.0}
modules_per_string = 1
tmy = read_csv("tmy.csv", parse_dates=["Timestamp"]).set_index("Timestamp")

## Object oriented

In [3]:
system = pvlib.pvsystem.PVSystem(
    surface_tilt=surface_tilt,
    surface_azimuth=surface_azimuth,
    module_parameters=module,
    inverter_parameters=inverter,
    modules_per_string=modules_per_string,
    module_type="glass_glass",
    racking_model="open_rack",
)
chain = pvlib.modelchain.ModelChain(
    system=system,
    location=pvlib.location.Location(
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
    ),
    aoi_model="physical",
    spectral_model="no_loss",
)
results = chain.run_model(weather=tmy)
results.ac.sum()

271015.87273861945

## Procedural

In [4]:
def no_spectral_loss_effective_irradiance(poa_direct, poa_diffuse, airmass_absolute, aoi, module):
    F1 = 1.0
    F2 = pvlib.iam.physical(aoi)
    FD = 1.0
    Ee = F1 * (poa_direct * F2 + FD * poa_diffuse)
    return Ee

index = tmy.index
solpos = pvlib.solarposition.get_solarposition(
    index,
    latitude,
    longitude,
    altitude,
)
dni_extra = pvlib.irradiance.get_extra_radiation(index)
pressure = pvlib.atmosphere.alt2pres(altitude)
relative_airmass = pvlib.atmosphere.get_relative_airmass(
    solpos['apparent_zenith'],
)
absolute_airmass = pvlib.atmosphere.get_absolute_airmass(
    relative_airmass,
    pressure,
)
aoi = pvlib.irradiance.aoi(
    surface_tilt,
    surface_azimuth,
    solpos["apparent_zenith"],
    solpos["azimuth"],
)
total_irradiance = pvlib.irradiance.get_total_irradiance(
    surface_tilt,
    surface_azimuth,
    solpos['apparent_zenith'],
    solpos['azimuth'],
    tmy['dni'],
    tmy['ghi'],
    tmy['dhi'],
    dni_extra=dni_extra,
    model='haydavies',
)
cell_temperature = pvlib.temperature.sapm_cell(
    total_irradiance['poa_global'],
    tmy["temp_air"],
    tmy["wind_speed"],
    **pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass'],
)
effective_irradiance = no_spectral_loss_effective_irradiance(
    total_irradiance['poa_direct'],
    total_irradiance['poa_diffuse'],
    absolute_airmass,
    aoi,module,
)
cec = pvlib.pvsystem.calcparams_cec(
    effective_irradiance,
    cell_temperature,
    alpha_sc=module["alpha_sc"],
    a_ref=module["a_ref"],
    I_L_ref=module["I_L_ref"],
    I_o_ref=module["I_o_ref"],
    R_sh_ref=module["R_sh_ref"],
    R_s=module["R_s"],
    Adjust=module["Adjust"],
)
dc = pvlib.pvsystem.singlediode(*cec)
ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)
ac.sum()

271018.0584959601